# 00 Demo: NL→SQL (prompt vs QLoRA vs ReAct)


Imports quick guide: we load schema helpers (`nl2sql.schema`), prompt builder/postprocess (`nl2sql.prompting`, `nl2sql.postprocess`), safe executor (`nl2sql.query_runner`), and model loader (`nl2sql.llm` or direct HF). These are small utilities we wrote to keep the notebooks thin.

Docs I leaned on: HF Transformers quantization (https://huggingface.co/docs/transformers/main_classes/quantization), PEFT/TRL (https://huggingface.co/docs/peft/, https://huggingface.co/docs/trl/), Cloud SQL connector + SQLAlchemy creator (https://cloud.google.com/sql/docs/mysql/connect-run, https://docs.sqlalchemy.org/en/20/core/engines.html#custom-dbapi-connect), ReAct (https://arxiv.org/abs/2210.03629), NL→SQL prompting survey (https://arxiv.org/abs/2410.06011).

Docs I leaned on: HF Transformers quantization (https://huggingface.co/docs/transformers/main_classes/quantization), PEFT/TRL (https://huggingface.co/docs/peft/, https://huggingface.co/docs/trl/), Cloud SQL connector + SQLAlchemy creator (https://cloud.google.com/sql/docs/mysql/connect-run, https://docs.sqlalchemy.org/en/20/core/engines.html#custom-dbapi-connect), ReAct (https://arxiv.org/abs/2210.03629).

This code just pins the Colab runtime so 4-bit Llama loads. Run once, restart, then skip.

## Setup (skip if runtime already prepared)
If you’re on a fresh Colab GPU, run this once, then restart runtime. Otherwise skip.

**Docs (setup):** HF Transformers quantization + BitsAndBytes (4-bit) https://huggingface.co/docs/transformers/main_classes/quantization, bnb https://github.com/TimDettmers/bitsandbytes.

In [ ]:

%%bash
set -e
export PIP_DEFAULT_TIMEOUT=120
pip uninstall -y torch torchvision torchaudio bitsandbytes triton transformers accelerate peft trl datasets numpy pandas fsspec requests google-auth || true
pip install -q --no-cache-dir --force-reinstall   numpy==1.26.4 pandas==2.2.1 fsspec==2024.5.0 requests==2.31.0 google-auth==2.43.0
pip install -q --no-cache-dir --force-reinstall   torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121   --index-url https://download.pytorch.org/whl/cu121
pip install -q --no-cache-dir --force-reinstall   bitsandbytes==0.43.3 triton==2.3.1   transformers==4.44.2 accelerate==0.33.0 peft==0.17.0 trl==0.9.6 datasets==2.20.0
print("Setup done. Restart runtime, then continue.")


This block is just auth/DB setup: HF token for Llama 3 and Cloud SQL connector + SQLAlchemy so the ClassicModels DB stays private.

## Auth and paths (explain to the audience)
- HF token: required to load gated Llama 3.
- DB creds: needed to hit ClassicModels via Cloud SQL Connector.
- Adapters: if present, we show fine-tuned behavior; if not, we show base model only.


**Docs (auth/DB):** Cloud SQL connector pattern https://cloud.google.com/sql/docs/mysql/connect-run; SQLAlchemy creator hook https://docs.sqlalchemy.org/en/20/core/engines.html#custom-dbapi-connect.

In [ ]:

import os
from getpass import getpass
from google.cloud.sql.connector import Connector
from google.oauth2.service_account import Credentials
from sqlalchemy import create_engine
from pathlib import Path

HF_TOKEN = os.getenv("HF_TOKEN") or getpass("Enter HF_TOKEN: ").strip()
INSTANCE_CONNECTION_NAME = os.getenv("INSTANCE_CONNECTION_NAME") or input("INSTANCE_CONNECTION_NAME: ")
DB_USER = os.getenv("DB_USER") or input("DB_USER: ")
DB_PASS = os.getenv("DB_PASS") or getpass("DB_PASS: ")
DB_NAME = os.getenv("DB_NAME") or "classicmodels"
GOOGLE_CREDS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
creds = Credentials.from_service_account_file(GOOGLE_CREDS) if GOOGLE_CREDS else None
connector = Connector(credentials=creds)

def getconn():
    return connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME,
    )
engine = create_engine("mysql+pymysql://", creator=getconn, future=True)
print("Engine ready")


Here we grab the schema summary (tables/columns) and a tiny demo slice (5 items). The schema text keeps the model from hallucinating tables.

## Load schema + small demo slice
We use the schema summary to ground prompts. We pick 5–10 items from the 200 test set for a fast demo.

**Docs (schema prompts):** NL→SQL schema-grounded prompting survey https://arxiv.org/abs/2410.06011; Spider-style listings.

In [ ]:

import json
from nl2sql.schema import build_schema_summary

SCHEMA_SUMMARY = build_schema_summary(engine, db_name=DB_NAME)
test_path = Path("data/classicmodels_test_200.json")
test_set = json.loads(test_path.read_text(encoding="utf-8"))
demo_set = test_set[:5]
print("Demo items:", len(demo_set))


Model load: HF 4-bit NF4 + BitsAndBytes. If adapters exist, we use them; otherwise base model. Deterministic decoding for repeatability.

## Load model (base or QLoRA adapters)
Refs: HF Transformers 4-bit NF4 + BitsAndBytes, PEFT QLoRA. Adapters are optional; if missing we use the base model and explain the difference live.


**Docs (model load):** HF 4-bit NF4 quantization https://huggingface.co/docs/transformers/main_classes/quantization; PEFT/QLoRA https://huggingface.co/docs/peft/.

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
ADAPTER_PATH = os.getenv("ADAPTER_PATH") or "results/adapters/qlora_classicmodels"

cc_major, _ = torch.cuda.get_device_capability(0) if torch.cuda.is_available() else (0,0)
compute_dtype = torch.bfloat16 if cc_major >= 8 else torch.float16

# Tokenizer
tok = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    torch_dtype=compute_dtype,
    device_map={"": 0} if torch.cuda.is_available() else None,
    token=HF_TOKEN,
)
base_model.generation_config.do_sample = False
base_model.generation_config.temperature = 1.0
base_model.generation_config.top_p = 1.0

from pathlib import Path
adapter_dir = Path(ADAPTER_PATH)
if adapter_dir.exists():
    model = PeftModel.from_pretrained(base_model, adapter_dir, token=HF_TOKEN)
    print("Loaded adapters from", adapter_dir)
else:
    model = base_model
    print("No adapters found; using base model")


Prompt-only demo: k=0 and k=3 on a few questions. We show prompt → SQL → execute, then VA/EX. Good for a quick before/after with adapters.

## Few examples: prompt-only vs QLoRA (k=0 and k=3)
We’ll run a handful of items with k=0 (no exemplars) and k=3 (few-shot). We show the prompt, the SQL, and execute it to prove it works.


**Docs (prompt/eval):** ICL patterns https://arxiv.org/abs/2005.14165; execution-based metrics (VA/EX) https://aclanthology.org/2020.emnlp-main.29/.

In [ ]:

from nl2sql.prompting import make_few_shot_messages
from nl2sql.postprocess import normalize_sql
from nl2sql.query_runner import QueryRunner
from nl2sql.eval import execution_accuracy

runner = QueryRunner(engine)

exemplars = demo_set[:3]

for k in [0,3]:
    print(f"== Demo k={k} ==")
    for item in demo_set:
        msgs = make_few_shot_messages(
            nlq=item['nlq'],
            schema_summary=SCHEMA_SUMMARY,
            exemplars=exemplars if k>0 else [],
            k=k,
        )
        prompt_preview = tok.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
        inputs = tok(prompt_preview, return_tensors="pt").to(model.device)
        out = model.generate(**inputs, max_new_tokens=256)
        text = tok.decode(out[0], skip_special_tokens=True)
        sql = normalize_sql(text)
        try:
            runner.run(sql)
            va=True
            ex = execution_accuracy(engine, sql, item['sql'])
        except Exception as e:
            va=False; ex=False; 
        print(f"Q: {item['nlq']}SQL: {sql}VA: {va} EX: {ex}")


Mini ReAct: propose SQL, run it via QueryRunner (SELECT-only), see error/result, refine once. Just to illustrate the idea without long runs.

## Tiny ReAct demo (1–2 steps)
We show how the agent proposes SQL, runs it, sees errors/results, and refines once. This is a toy loop to illustrate the idea.


In [ ]:

from nl2sql.query_runner import QueryRunner
runner = QueryRunner(engine)

PROMPT_INSTR = "You are an expert SQL agent. Think briefly, propose one SELECT query, then adjust if there is an error."

def react_once(nlq, schema_text):
    history=[]
    observation=""
    for step in range(2):
        prompt = f"{PROMPT_INSTR}Schema:{schema_text}Previous observation: {observation}Question: {nlq}SQL:"
        inputs = tok(prompt, return_tensors="pt").to(model.device)
        out = model.generate(**inputs, max_new_tokens=192)
        text = tok.decode(out[0], skip_special_tokens=True)
        sql = normalize_sql(text)
        try:
            rows = runner.run(sql)
            observation=f"SUCCESS ({len(rows)} rows)"
            return sql, observation
        except Exception as e:
            observation=f"ERROR: {e}"
            history.append((sql, observation))
    return sql, observation

sample = demo_set[0]
sql, obs = react_once(sample['nlq'], SCHEMA_SUMMARY)
print(f"Q: {sample['nlq']}SQL: {sql}Obs: {obs}")


Recap: cite the full 200-item results from saved JSONs instead of rerunning heavy evals.

## Cite full results without rerunning
We already ran the full 200-item evaluations. Headline numbers (from saved JSONs in `results/qlora/`):
- Prompt few-shot EX: ~0.25–0.33
- QLoRA k=3 EX: ~0.38 (VA ~0.87)
- QLoRA k=0 EX: ~0.065 (needs agentic/TS)
Use these in the demo; no need to recompute live.
